In [5]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report



In [35]:
df=sns.load_dataset('titanic')

In [36]:
#impute missing values
from sklearn.impute import KNNImputer



In [37]:
#impute missing values
imputer = KNNImputer(n_neighbors=5)
df['fare'] = imputer.fit_transform(df[['fare']])
df['age'] = imputer.fit_transform(df[['age']])



In [38]:
#impute missing values in embark and emabr tow
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])
df['embark_town'] = df['embark_town'].fillna(df['embark_town'].mode()[0])

In [39]:
#drop deck column
df.drop(['deck','alive'],axis=1,inplace=True)

In [40]:
df.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alone          0
dtype: int64

In [41]:
#convert each categorical columns to category
cat_columns=df.select_dtypes(include=['object','category']).columns

#add this to new columns
df[cat_columns]=df[cat_columns].astype('category')

In [42]:
#split data
#drop survived and alive

X=df.drop(['survived'],axis=1)
y=df['survived']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [43]:
#run the catboost
clf = CatBoostClassifier(iterations=1000, learning_rate=0.01, depth=6, loss_function='Logloss',eval_metric='Accuracy',random_seed=42,verbose=False)
clf.fit(X_train, y_train,cat_features=cat_columns.to_list())
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8212290502793296
Confusion Matrix:
 [[94 11]
 [21 53]]
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.90      0.85       105
           1       0.83      0.72      0.77        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179

